In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from core.embedding_models import FastTextEmbeddingModel
from core.models import LinearRegressionModel, CatboostRegressionModel

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
df = pd.read_csv("../data/data_vacancies.csv")
df.head()

,id,custom_position,schedule,salary_from,salary_to,salary_pay_type,offer_education_id,education_name,education_is_base,education_order_num,city_id,list_regions,work_skills,tags_id
0,48202096,Сварщик-сборщик,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['сварочные работы', 'сборка изделий по чертеж...",NaN
1,48202097,Сварщик-монтажник,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['монтажные работы', 'строительные работы', 'э...",NaN
2,48202098,Слесарь-сборщик,полный рабочий день,60000,80000,NaN,0,любое,True,0,2,[4],"['работа на фрезерных станках', 'слесарный рем...",NaN
3,48202356,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,1,[3],"['комплектация товара', 'маркировка', 'стрессо...","[6, 9]"
4,48202357,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,57,"[181, 182, 183, 185, 186, 187, 188, 189, 190, ...","['маркировка', 'стрессоустойчивость', 'погрузо...","[6, 9]"


In [25]:
df["salary_mean"] = (df.salary_from + df.salary_to) / 2
df["target"] = df.salary_mean
df = df[df.salary_mean < df.salary_mean.quantile(0.95)]

In [5]:
fasttext_embedding_model = FastTextEmbeddingModel()

In [26]:
lr_model = LinearRegressionModel(embedding_model=fasttext_embedding_model)

In [27]:
cols_to_concat = ["custom_position"]
df["combined_string"] = df[cols_to_concat].agg("\n".join, axis=1)

df["emb"] = df.combined_string.apply(lambda x: fasttext_embedding_model.generate(x))

In [28]:
lr_model.train(df)

Model trained
Test score is 16277.157743659482


In [29]:
cb_model = CatboostRegressionModel(embedding_model=fasttext_embedding_model)


In [31]:
cb_model.train(df)

Learning rate set to 0.062656
0:	learn: 26545.2642466	total: 116ms	remaining: 1m 56s
1:	learn: 26191.5897309	total: 133ms	remaining: 1m 6s
2:	learn: 25880.7847799	total: 144ms	remaining: 48s
3:	learn: 25613.8836302	total: 155ms	remaining: 38.7s
4:	learn: 25351.0256519	total: 167ms	remaining: 33.2s
5:	learn: 25102.9987668	total: 178ms	remaining: 29.5s
6:	learn: 24864.4472099	total: 189ms	remaining: 26.9s
7:	learn: 24644.8925483	total: 204ms	remaining: 25.3s
8:	learn: 24442.8066027	total: 218ms	remaining: 23.9s
9:	learn: 24250.6283971	total: 235ms	remaining: 23.3s
10:	learn: 24074.7795892	total: 250ms	remaining: 22.5s
11:	learn: 23909.5787534	total: 261ms	remaining: 21.5s
12:	learn: 23757.5890215	total: 273ms	remaining: 20.7s
13:	learn: 23588.7282334	total: 286ms	remaining: 20.2s
14:	learn: 23445.7402717	total: 299ms	remaining: 19.7s
15:	learn: 23294.9179759	total: 311ms	remaining: 19.1s
16:	learn: 23161.1251233	total: 323ms	remaining: 18.7s
17:	learn: 23015.4805300	total: 335ms	remainin